# Download and prepare the data. Explain the procedure.

### Project description

To produce gold a row ore goes in 3 steps where the second and third are part of step calls purification and the first calls flotation. In every step a raw material enters and two streams leave. One is with the concentrate and this is the main process line and where most of the gold stays and the second calls tails and there are all the leftover that will leave the process and if any gold passed there this is a loss and it will cause a reduction in the recovery.

### Import needed libraries

In [290]:
# Data tools
import pandas as pd
import numpy as np

# Graphics and display
from IPython.core.interactiveshell import InteractiveShell
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
%matplotlib inline

# Ml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

# Statistics
from scipy import stats

print('Project libraries has been successfully been imported!')

Project libraries has been successfully been imported!


 ### Set environment variables¶

In [291]:
# set to display all output not only print() or last output
InteractiveShell.ast_node_interactivity = "all"        

## Open the files and look into the data.

In [292]:
try:
    data_full = pd.read_csv('gold_recovery_full.csv')
    data_test = pd.read_csv('gold_recovery_test.csv')
    data_train = pd.read_csv('gold_recovery_train.csv')
except:
    data_full = pd.read_csv('/datasets/gold_recovery_full.csv')
    data_test = pd.read_csv('/datasets/gold_recovery_test.csv')
    data_train = pd.read_csv('/datasets/gold_recovery_train.csv')
    
print('Data has been read correctly!')

Data has been read correctly!


### Looking at the dataset

### Headers of the data

There are many features in this project and to easily tells what is in each one there is a formal naming system:

[stage].[parameter_type].[parameter_name]

Example: rougher.input.feed_ag


### Data describe

In [293]:
# Checking 0's
def zero_check(df):
    for i in df:
        print(i,len(df[df[i]==0]))
        
# function to determine if columns in file have null values        
def get_percent_of_na(df, num):
    count = 0
    df = df.copy()
    s = (df.isna().sum() / df.shape[0])
    for column, percent in zip(s.index, s.values):
        num_of_nulls = df[column].isna().sum()
        if num_of_nulls == 0:
            continue
        else:
            count += 1
        print('{} has {} nulls, which is {:.{}%} percent of Nulls'.format(column, num_of_nulls, percent, num))
    if count != 0:
        print("\033[1m" + 'There are {} columns with NA.'.format(count) + "\033[0m")
    else:
        print()
        print("\033[1m" + 'There are no columns with NA.' + "\033[0m")       
        
# function to display general information about the dataset
def general_info(df):
    print("\033[1m" + "\033[0m")
    display(pd.concat([df.dtypes, df.count(),df.isna().sum(),df.isna().sum()/len(df)], keys=['type','count','na','na%'],
                      axis=1))
    print()
    print("\033[1m" + 'Head:')  
    display(df.head())
    print()
    print("\033[1m" + 'Tail:')
    display(df.tail())
    print()
    print("\033[1m" + 'Info:')
    print()
    display(df.info())
    print()
    print("\033[1m" + 'Describe:')
    print()
    display(df.describe())
    print()
    print("\033[1m" + 'Describe include: all :')
    print()
    display(df.describe(include='all'))
    print()
    print("\033[1m" + 'nulls in the columns:')
    print()
    display(get_percent_of_na(df, 4))  # check this out
    print()
    print("\033[1m" + 'Zeros in the columns:') 
    print()
    display(zero_check(df))
    print()
    print("\033[1m" + 'Shape:', df.shape)
    print()
    print()
    print('Duplicated:',"\033[1m" + 'We have {} duplicated rows\n'.format(df.duplicated().sum()) + "\033[0m")
    print()
    print("\033[1m" + 'Dtypes:')  
    display(df.dtypes)
    print()

#### data_full

In [294]:
#print our info data
print('information about "data_full" dataset:')
general_info(data_full)

information about "data_full" dataset:



,type,count,na,na%
date,object,22716,0,0.000000
final.output.concentrate_ag,float64,22627,89,0.003918
final.output.concentrate_pb,float64,22629,87,0.003830
final.output.concentrate_sol,float64,22331,385,0.016948
final.output.concentrate_au,float64,22630,86,0.003786
...,...,...,...,...
secondary_cleaner.state.floatbank5_a_level,float64,22615,101,0.004446
secondary_cleaner.state.floatbank5_b_air,float64,22615,101,0.004446
secondary_cleaner.state.floatbank5_b_level,float64,22616,100,0.004402
secondary_cleaner.state.floatbank6_a_air,float64,22597,119,0.005239



Head:


,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2016-01-15 00:00:00,6.055403,9.889648,5.507324,42.192020,70.541216,10.411962,0.895447,16.904297,2.143149,...,14.016835,-502.488007,12.099931,-504.715942,9.925633,-498.310211,8.079666,-500.470978,14.151341,-605.841980
1,2016-01-15 01:00:00,6.029369,9.968944,5.257781,42.701629,69.266198,10.462676,0.927452,16.634514,2.224930,...,13.992281,-505.503262,11.950531,-501.331529,10.039245,-500.169983,7.984757,-500.582168,13.998353,-599.787184
2,2016-01-15 02:00:00,6.055926,10.213995,5.383759,42.657501,68.116445,10.507046,0.953716,16.208849,2.257889,...,14.015015,-502.520901,11.912783,-501.133383,10.070913,-500.129135,8.013877,-500.517572,14.028663,-601.427363
3,2016-01-15 03:00:00,6.047977,9.977019,4.858634,42.689819,68.347543,10.422762,0.883763,16.532835,2.146849,...,14.036510,-500.857308,11.999550,-501.193686,9.970366,-499.201640,7.977324,-500.255908,14.005551,-599.996129
4,2016-01-15 04:00:00,6.148599,10.142511,4.939416,42.774141,66.927016,10.360302,0.792826,16.525686,2.055292,...,14.027298,-499.838632,11.953070,-501.053894,9.925709,-501.686727,7.894242,-500.356035,13.996647,-601.496691



Tail:


...

None


Zeros in the columns:

date 0
final.output.concentrate_ag 1613
final.output.concentrate_pb 1613
final.output.concentrate_sol 1613
final.output.concentrate_au 1613
final.output.recovery 151
final.output.tail_ag 1950
final.output.tail_pb 1950
final.output.tail_sol 1950
final.output.tail_au 1950
primary_cleaner.input.sulfate 0
primary_cleaner.input.depressant 57
primary_cleaner.input.feed_size 0
primary_cleaner.input.xanthate 0
primary_cleaner.output.concentrate_ag 1626
primary_cleaner.output.concentrate_pb 1626
primary_cleaner.output.concentrate_sol 1626
primary_cleaner.output.concentrate_au 1626
primary_cleaner.output.tail_ag 1953
primary_cleaner.output.tail_pb 1953
primary_cleaner.output.tail_sol 1953
primary_cleaner.output.tail_au 1953
primary_cleaner.state.floatbank8_a_air 349
primary_cleaner.state.floatbank8_a_level 0
primary_cleaner.state.floatbank8_b_air 345
primary_cleaner.state.floatbank8_b_level 0
primary_cleaner.state.floatbank8_c_air 330
primary_cleaner.state.floatbank8_c_le

None


Shape: (22716, 87)


Duplicated: We have 0 duplicated rows


Dtypes:


date                                           object
final.output.concentrate_ag                   float64
final.output.concentrate_pb                   float64
final.output.concentrate_sol                  float64
final.output.concentrate_au                   float64
                                               ...   
secondary_cleaner.state.floatbank5_a_level    float64
secondary_cleaner.state.floatbank5_b_air      float64
secondary_cleaner.state.floatbank5_b_level    float64
secondary_cleaner.state.floatbank6_a_air      float64
secondary_cleaner.state.floatbank6_a_level    float64
Length: 87, dtype: object

notes
- There are 87 columns in the data
- All values are floats
- Some values are negative. They are reading of levels indicators. Since it's relative information it make sense
- There are 22716 observations in the data
- Almost all columns contains Non values
- Part of the columns contains some zeroes
- No duplicates rows


data_train['new_rougher_recovery'] = (
    data_train['rougher.output.concentrate_au'] * (
        data_train['rougher.input.feed_au'] - data_train['rougher.output.tail_au']) ) \
/ (data_train['rougher.input.feed_au'] * (
    data_train['rougher.output.concentrate_au'] - data_train['rougher.output.tail_au']) )*100

#### data_train

In [295]:
#print our info data
print('information about "data_train" dataset:')
general_info(data_train)

information about "data_train" dataset:



,type,count,na,na%
date,object,16860,0,0.000000
final.output.concentrate_ag,float64,16788,72,0.004270
final.output.concentrate_pb,float64,16788,72,0.004270
final.output.concentrate_sol,float64,16490,370,0.021945
final.output.concentrate_au,float64,16789,71,0.004211
...,...,...,...,...
secondary_cleaner.state.floatbank5_a_level,float64,16775,85,0.005042
secondary_cleaner.state.floatbank5_b_air,float64,16775,85,0.005042
secondary_cleaner.state.floatbank5_b_level,float64,16776,84,0.004982
secondary_cleaner.state.floatbank6_a_air,float64,16757,103,0.006109



Head:


,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2016-01-15 00:00:00,6.055403,9.889648,5.507324,42.192020,70.541216,10.411962,0.895447,16.904297,2.143149,...,14.016835,-502.488007,12.099931,-504.715942,9.925633,-498.310211,8.079666,-500.470978,14.151341,-605.841980
1,2016-01-15 01:00:00,6.029369,9.968944,5.257781,42.701629,69.266198,10.462676,0.927452,16.634514,2.224930,...,13.992281,-505.503262,11.950531,-501.331529,10.039245,-500.169983,7.984757,-500.582168,13.998353,-599.787184
2,2016-01-15 02:00:00,6.055926,10.213995,5.383759,42.657501,68.116445,10.507046,0.953716,16.208849,2.257889,...,14.015015,-502.520901,11.912783,-501.133383,10.070913,-500.129135,8.013877,-500.517572,14.028663,-601.427363
3,2016-01-15 03:00:00,6.047977,9.977019,4.858634,42.689819,68.347543,10.422762,0.883763,16.532835,2.146849,...,14.036510,-500.857308,11.999550,-501.193686,9.970366,-499.201640,7.977324,-500.255908,14.005551,-599.996129
4,2016-01-15 04:00:00,6.148599,10.142511,4.939416,42.774141,66.927016,10.360302,0.792826,16.525686,2.055292,...,14.027298,-499.838632,11.953070,-501.053894,9.925709,-501.686727,7.894242,-500.356035,13.996647,-601.496691



Tail:


...

None


Zeros in the columns:

date 0
final.output.concentrate_ag 1263
final.output.concentrate_pb 1263
final.output.concentrate_sol 1263
final.output.concentrate_au 1263
final.output.recovery 89
final.output.tail_ag 1658
final.output.tail_pb 1658
final.output.tail_sol 1658
final.output.tail_au 1658
primary_cleaner.input.sulfate 0
primary_cleaner.input.depressant 57
primary_cleaner.input.feed_size 0
primary_cleaner.input.xanthate 0
primary_cleaner.output.concentrate_ag 1230
primary_cleaner.output.concentrate_pb 1230
primary_cleaner.output.concentrate_sol 1230
primary_cleaner.output.concentrate_au 1230
primary_cleaner.output.tail_ag 1549
primary_cleaner.output.tail_pb 1549
primary_cleaner.output.tail_sol 1549
primary_cleaner.output.tail_au 1549
primary_cleaner.state.floatbank8_a_air 256
primary_cleaner.state.floatbank8_a_level 0
primary_cleaner.state.floatbank8_b_air 257
primary_cleaner.state.floatbank8_b_level 0
primary_cleaner.state.floatbank8_c_air 282
primary_cleaner.state.floatbank8_c_lev

None


Shape: (16860, 87)


Duplicated: We have 0 duplicated rows


Dtypes:


date                                           object
final.output.concentrate_ag                   float64
final.output.concentrate_pb                   float64
final.output.concentrate_sol                  float64
final.output.concentrate_au                   float64
                                               ...   
secondary_cleaner.state.floatbank5_a_level    float64
secondary_cleaner.state.floatbank5_b_air      float64
secondary_cleaner.state.floatbank5_b_level    float64
secondary_cleaner.state.floatbank6_a_air      float64
secondary_cleaner.state.floatbank6_a_level    float64
Length: 87, dtype: object

notes
- There are 87 columns in the data
- All values are floats
- Some values are negative. They are reading of levels indicators. Since it's relative information it make sense
- There are 16860 observations in the data
- Almost all columns contains Non values
- Part of the columns contains some zeroes
- No duplicates rows

1.1.3.2  data_test

In [296]:
#print our info data
print('information about "data_test" dataset:')
general_info(data_test)

information about "data_test" dataset:



,type,count,na,na%
date,object,5856,0,0.000000
primary_cleaner.input.sulfate,float64,5554,302,0.051571
primary_cleaner.input.depressant,float64,5572,284,0.048497
primary_cleaner.input.feed_size,float64,5856,0,0.000000
primary_cleaner.input.xanthate,float64,5690,166,0.028347
primary_cleaner.state.floatbank8_a_air,float64,5840,16,0.002732
primary_cleaner.state.floatbank8_a_level,float64,5840,16,0.002732
primary_cleaner.state.floatbank8_b_air,float64,5840,16,0.002732
primary_cleaner.state.floatbank8_b_level,float64,5840,16,0.002732
primary_cleaner.state.floatbank8_c_air,float64,5840,16,0.002732



Head:


,date,primary_cleaner.input.sulfate,primary_cleaner.input.depressant,primary_cleaner.input.feed_size,primary_cleaner.input.xanthate,primary_cleaner.state.floatbank8_a_air,primary_cleaner.state.floatbank8_a_level,primary_cleaner.state.floatbank8_b_air,primary_cleaner.state.floatbank8_b_level,primary_cleaner.state.floatbank8_c_air,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
0,2016-09-01 00:59:59,210.800909,14.993118,8.080000,1.005021,1398.981301,-500.225577,1399.144926,-499.919735,1400.102998,...,12.023554,-497.795834,8.016656,-501.289139,7.946562,-432.317850,4.872511,-500.037437,26.705889,-499.709414
1,2016-09-01 01:59:59,215.392455,14.987471,8.080000,0.990469,1398.777912,-500.057435,1398.055362,-499.778182,1396.151033,...,12.058140,-498.695773,8.130979,-499.634209,7.958270,-525.839648,4.878850,-500.162375,25.019940,-499.819438
2,2016-09-01 02:59:59,215.259946,12.884934,7.786667,0.996043,1398.493666,-500.868360,1398.860436,-499.764529,1398.075709,...,11.962366,-498.767484,8.096893,-500.827423,8.071056,-500.801673,4.905125,-499.828510,24.994862,-500.622559
3,2016-09-01 03:59:59,215.336236,12.006805,7.640000,0.863514,1399.618111,-498.863574,1397.440120,-499.211024,1400.129303,...,12.033091,-498.350935,8.074946,-499.474407,7.897085,-500.868509,4.931400,-499.963623,24.948919,-498.709987
4,2016-09-01 04:59:59,199.099327,10.682530,7.530000,0.805575,1401.268123,-500.808305,1398.128818,-499.504543,1402.172226,...,12.025367,-500.786497,8.054678,-500.397500,8.107890,-509.526725,4.957674,-500.360026,25.003331,-500.856333



Tail:


...

None


Zeros in the columns:

date 0
primary_cleaner.input.sulfate 0
primary_cleaner.input.depressant 0
primary_cleaner.input.feed_size 0
primary_cleaner.input.xanthate 0
primary_cleaner.state.floatbank8_a_air 93
primary_cleaner.state.floatbank8_a_level 0
primary_cleaner.state.floatbank8_b_air 88
primary_cleaner.state.floatbank8_b_level 0
primary_cleaner.state.floatbank8_c_air 48
primary_cleaner.state.floatbank8_c_level 0
primary_cleaner.state.floatbank8_d_air 48
primary_cleaner.state.floatbank8_d_level 0
rougher.input.feed_ag 369
rougher.input.feed_pb 369
rougher.input.feed_rate 0
rougher.input.feed_size 0
rougher.input.feed_sol 369
rougher.input.feed_au 369
rougher.input.floatbank10_sulfate 0
rougher.input.floatbank10_xanthate 0
rougher.input.floatbank11_sulfate 0
rougher.input.floatbank11_xanthate 0
rougher.state.floatbank10_a_air 0
rougher.state.floatbank10_a_level 0
rougher.state.floatbank10_b_air 0
rougher.state.floatbank10_b_level 0
rougher.state.floatbank10_c_air 0
rougher.state.floa

None


Shape: (5856, 53)


Duplicated: We have 0 duplicated rows


Dtypes:


date                                           object
primary_cleaner.input.sulfate                 float64
primary_cleaner.input.depressant              float64
primary_cleaner.input.feed_size               float64
primary_cleaner.input.xanthate                float64
primary_cleaner.state.floatbank8_a_air        float64
primary_cleaner.state.floatbank8_a_level      float64
primary_cleaner.state.floatbank8_b_air        float64
primary_cleaner.state.floatbank8_b_level      float64
primary_cleaner.state.floatbank8_c_air        float64
primary_cleaner.state.floatbank8_c_level      float64
primary_cleaner.state.floatbank8_d_air        float64
primary_cleaner.state.floatbank8_d_level      float64
rougher.input.feed_ag                         float64
rougher.input.feed_pb                         float64
rougher.input.feed_rate                       float64
rougher.input.feed_size                       float64
rougher.input.feed_sol                        float64
rougher.input.feed_au       

notes
- There are 53 columns in the data
- All values are floats
- Some values are negative. They are reading of levels indicators. Since it's relative information it make sense
- There are 5856 observations in the data
- Almost all columns contains Non values
- Part of the columns contains some zeroes
- No duplicates rows

## Check that recovery is calculated correctly. Using the training set, calculate recovery for the rougher.output.recovery feature. Find the MAE between your calculations and the feature values. Provide findings.


In this section I will:

- Create new column with my calculation for rougher.output.recovery
- Calculate MAE for my column (prediction) vs the original (target)
- For this section use only rows without Non in the relevant column:
rougher.output.concentrate_au , rougher.input.feed_au ,  rougher.output.tail_au , rougher.output.recovery


In [297]:
# drop nan in all relevant columns in the training set for this calculation 
data_train_remove_nan = data_train.dropna(
    subset=['rougher.output.concentrate_au',
    'rougher.input.feed_au',
    'rougher.output.tail_au',
    'rougher.output.recovery'])

In [298]:
data_train_remove_nan[['rougher.output.concentrate_au',
    'rougher.input.feed_au',
    'rougher.output.tail_au',
    'rougher.output.recovery']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14287 entries, 0 to 16859
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   rougher.output.concentrate_au  14287 non-null  float64
 1   rougher.input.feed_au          14287 non-null  float64
 2   rougher.output.tail_au         14287 non-null  float64
 3   rougher.output.recovery        14287 non-null  float64
dtypes: float64(4)
memory usage: 558.1 KB


ok

In [299]:
data_train_remove_nan['new_rougher.output.recovery'] = ((
    data_train_remove_nan['rougher.output.concentrate_au'] * (
        data_train_remove_nan['rougher.input.feed_au'] - data_train_remove_nan['rougher.output.tail_au']) ) \
/ (data_train_remove_nan['rougher.input.feed_au'] * (
    data_train_remove_nan['rougher.output.concentrate_au'] - data_train_remove_nan['rougher.output.tail_au']) ))*100

In [300]:
mae = mean_absolute_error(y_true=data_train_remove_nan['rougher.output.recovery'],
                    y_pred=data_train_remove_nan['new_rougher.output.recovery'])

In [301]:
print('The MAE for the calculation of rougher recovery is: {:e}' .format(mae))

The MAE for the calculation of rougher recovery is: 9.303416e-15


The calculation predict exactly the recovery in the data. 

## Analyze the features not available in the test set. What are these parameters? What is their type?

In this section I will:
- Create a data frame using full data as a base with all features that are absent from the test.
- Check the info of this data frame
- Provide findings in text


In [302]:
# list of missing columns in the data_test include the date

# connect all columns names
column_list_with_duplicates  = list(data_full.columns.values) + list(data_test.columns.values)
#drop duplicates and leve date
only_missing_columns_list = [
    x for x in column_list_with_duplicates if column_list_with_duplicates.count(x)==1 or x == 'date']
# remove last value - it is date . The first value is also date
only_missing_columns_list = only_missing_columns_list[:-1]

In [303]:
# create data frame with only missing column
data_full_34_columns = data_full[only_missing_columns_list]

In [304]:
# add the missing 34 columns to the data_test to have all the removed features back in the test set
data_test_with_missing = data_test.merge(right=data_full_34_columns, how='left' ,on='date')

# create df of test set with only the features that was missing
data_test_only_missing = data_test_with_missing[only_missing_columns_list]

In [305]:
# print the type of the features that are not available in the test set
data_test_only_missing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5856 entries, 0 to 5855
Data columns (total 35 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                5856 non-null   object 
 1   final.output.concentrate_ag                         5839 non-null   float64
 2   final.output.concentrate_pb                         5841 non-null   float64
 3   final.output.concentrate_sol                        5841 non-null   float64
 4   final.output.concentrate_au                         5841 non-null   float64
 5   final.output.recovery                               5414 non-null   float64
 6   final.output.tail_ag                                5839 non-null   float64
 7   final.output.tail_pb                                5839 non-null   float64
 8   final.output.tail_sol                               5730 non-null   float64
 9

In [306]:
# print the info on the data_test
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5856 entries, 0 to 5855
Data columns (total 53 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   date                                        5856 non-null   object 
 1   primary_cleaner.input.sulfate               5554 non-null   float64
 2   primary_cleaner.input.depressant            5572 non-null   float64
 3   primary_cleaner.input.feed_size             5856 non-null   float64
 4   primary_cleaner.input.xanthate              5690 non-null   float64
 5   primary_cleaner.state.floatbank8_a_air      5840 non-null   float64
 6   primary_cleaner.state.floatbank8_a_level    5840 non-null   float64
 7   primary_cleaner.state.floatbank8_b_air      5840 non-null   float64
 8   primary_cleaner.state.floatbank8_b_level    5840 non-null   float64
 9   primary_cleaner.state.floatbank8_c_air      5840 non-null   float64
 10  primary_clea

### Notes

- All the parameters are floats
- The removed features include all calculation features
- The test set contains only the inputs to the process and process state parameters - **not any output** so from the test set all process output was removed


- Since the features of the training and the set must match to use an ML model a decision of how to treat the missing column must be taken. I will use the training without the features that are missing in the test set. The model will be base on the process input and process state values only!


## Perform data preprocessing

### Retain the originals data frames with the addition of _raw to their name

In [307]:
data_full_raw = data_full.copy(deep=True)
data_test_raw = data_test.copy(deep=True)
data_train_raw = data_train.copy(deep=True)

### EDA to determine how to treat the data

In [308]:
# Check how many nan in this set
data_full_na = data_full.isna().sum().to_frame()

fig = px.bar(data_full_na, orientation='h', width=900, height=2000)
fig.show()

There are lots of missing values in the data. I will observe the reading of the features. The problem is that producing scatter plots  and histograms of all the observations is to heavy. The solution I choose is performing downsampling. I will leave only 1 out of every 100 observations and do it systematically while reading from the csv using the skiprows argument. I will create new data frames for the full csv this way.

In [309]:
# a function that return true if the index modulue for 100 in the opend dataframe is not 0. 
# This way it will skip all rows beside the one their index modulue 100 is 0.
def skip(index):
    if index % 100 != 0:
       return True

In [310]:
try:
    data_full_downsample = pd.read_csv('gold_recovery_full.csv', skiprows= lambda x: skip(x))
except:
    data_full_downsample = pd.read_csv('/datasets/gold_recovery_full.csv', skiprows= lambda x: skip(x))
    
print('Data has been read correctly!')

Data has been read correctly!


In [311]:
# produce scatter for one of ~10 of the downsample features
skipping_interval = int(data_full_downsample.shape[1]/10)
counter = 1
for column in data_full_downsample.columns:
    if counter % skipping_interval == 0:
        print(f'{column} scatter and histogram')
        px.scatter(data_full_downsample, x='date', y=column).show()
        px.histogram(data_full_downsample, x=column).show()
        print()
        print()
    
    counter += 1

final.output.tail_pb scatter and histogram




primary_cleaner.output.concentrate_pb scatter and histogram




primary_cleaner.state.floatbank8_a_level scatter and histogram




rougher.calculation.floatbank10_sulfate_to_au_feed scatter and histogram




rougher.input.feed_au scatter and histogram




rougher.output.concentrate_au scatter and histogram




rougher.state.floatbank10_b_air scatter and histogram




rougher.state.floatbank10_f_air scatter and histogram




secondary_cleaner.state.floatbank2_b_air scatter and histogram




secondary_cleaner.state.floatbank4_b_air scatter and histogram


- When observing the data we can see that the values are generally represent continues process and therefore if we want to replace nan we better do it base on interpolate values. 
- There are some extreme values. My guess is that this is a noise that comes from the analysers. I will leave them since this represents the process and in any given test set and future predictions they will appear and I will have to let the model figure out how to treat them.
- Values are in different scale. This will interfere any regression model. Therefore I will scale my features.

#### Handling Missing Data

The strategy I will choose is replacing with interpolate values because the values produced from a physical process and they tend to be relate and similar to the values that come before and after.

#### hold

# increase interpolation gap untill there are no nan
sum_of_na = 1
i = 1
while sum_of_na > 0:
    temp_data_train = data_train.interpolate(method='linear' ,limit=i)
    print(f'The limit of interpolate values is: {i}')
    sum_of_na = temp_data_train.isna().sum().sum()
    print(f'The sum of na in the data frame is {sum_of_na}')    
    i += 1

data_train = temp_data_train

In [312]:
data_train = data_train.interpolate(method='linear' ,limit=287)

Do the same for the test set

#### hold

# increase interpolation gap untill there are no nan
sum_of_na = 1
i = 1
while sum_of_na > 0:
    temp_data_test = data_test.interpolate(method='linear' ,limit=i)
    print(f'The limit of interpolate values is: {i}')
    sum_of_na = temp_data_test.isna().sum().sum()
    print(f'The sum of na in the data frame is {sum_of_na}')    
    i += 1

data_test = temp_data_test

In [313]:
data_test = data_test.interpolate(method='linear' ,limit=109)

Do the same for the full set

#### hold

sum_of_na = 1
i = 1
while sum_of_na > 0:
    temp_data_full = data_full.interpolate(method='linear' ,limit=i)
    print(f'The limit of interpolate values is: {i}')
    sum_of_na = temp_data_full.isna().sum().sum()
    print(f'The sum of na in the data frame is {sum_of_na}')    
    i += 1

data_full = temp_data_full

In [314]:
data_full = data_full.interpolate(method='linear' ,limit=287) 

#### hold hold

### Feature Scaling

#### create numerical columns for the scaling

numeric_full_train = ['final.output.concentrate_ag', 'final.output.concentrate_pb',
       'final.output.concentrate_sol', 'final.output.concentrate_au',
       'final.output.recovery', 'final.output.tail_ag', 'final.output.tail_pb',
       'final.output.tail_sol', 'final.output.tail_au',
       'primary_cleaner.input.sulfate', 'primary_cleaner.input.depressant',
       'primary_cleaner.input.feed_size', 'primary_cleaner.input.xanthate',
       'primary_cleaner.output.concentrate_ag',
       'primary_cleaner.output.concentrate_pb',
       'primary_cleaner.output.concentrate_sol',
       'primary_cleaner.output.concentrate_au',
       'primary_cleaner.output.tail_ag', 'primary_cleaner.output.tail_pb',
       'primary_cleaner.output.tail_sol', 'primary_cleaner.output.tail_au',
       'primary_cleaner.state.floatbank8_a_air',
       'primary_cleaner.state.floatbank8_a_level',
       'primary_cleaner.state.floatbank8_b_air',
       'primary_cleaner.state.floatbank8_b_level',
       'primary_cleaner.state.floatbank8_c_air',
       'primary_cleaner.state.floatbank8_c_level',
       'primary_cleaner.state.floatbank8_d_air',
       'primary_cleaner.state.floatbank8_d_level',
       'rougher.calculation.sulfate_to_au_concentrate',
       'rougher.calculation.floatbank10_sulfate_to_au_feed',
       'rougher.calculation.floatbank11_sulfate_to_au_feed',
       'rougher.calculation.au_pb_ratio', 'rougher.input.feed_ag',
       'rougher.input.feed_pb', 'rougher.input.feed_rate',
       'rougher.input.feed_size', 'rougher.input.feed_sol',
       'rougher.input.feed_au', 'rougher.input.floatbank10_sulfate',
       'rougher.input.floatbank10_xanthate',
       'rougher.input.floatbank11_sulfate',
       'rougher.input.floatbank11_xanthate', 'rougher.output.concentrate_ag',
       'rougher.output.concentrate_pb', 'rougher.output.concentrate_sol',
       'rougher.output.concentrate_au', 'rougher.output.recovery',
       'rougher.output.tail_ag', 'rougher.output.tail_pb',
       'rougher.output.tail_sol', 'rougher.output.tail_au',
       'rougher.state.floatbank10_a_air', 'rougher.state.floatbank10_a_level',
       'rougher.state.floatbank10_b_air', 'rougher.state.floatbank10_b_level',
       'rougher.state.floatbank10_c_air', 'rougher.state.floatbank10_c_level',
       'rougher.state.floatbank10_d_air', 'rougher.state.floatbank10_d_level',
       'rougher.state.floatbank10_e_air', 'rougher.state.floatbank10_e_level',
       'rougher.state.floatbank10_f_air', 'rougher.state.floatbank10_f_level',
       'secondary_cleaner.output.tail_ag', 'secondary_cleaner.output.tail_pb',
       'secondary_cleaner.output.tail_sol', 'secondary_cleaner.output.tail_au',
       'secondary_cleaner.state.floatbank2_a_air',
       'secondary_cleaner.state.floatbank2_a_level',
       'secondary_cleaner.state.floatbank2_b_air',
       'secondary_cleaner.state.floatbank2_b_level',
       'secondary_cleaner.state.floatbank3_a_air',
       'secondary_cleaner.state.floatbank3_a_level',
       'secondary_cleaner.state.floatbank3_b_air',
       'secondary_cleaner.state.floatbank3_b_level',
       'secondary_cleaner.state.floatbank4_a_air',
       'secondary_cleaner.state.floatbank4_a_level',
       'secondary_cleaner.state.floatbank4_b_air',
       'secondary_cleaner.state.floatbank4_b_level',
       'secondary_cleaner.state.floatbank5_a_air',
       'secondary_cleaner.state.floatbank5_a_level',
       'secondary_cleaner.state.floatbank5_b_air',
       'secondary_cleaner.state.floatbank5_b_level',
       'secondary_cleaner.state.floatbank6_a_air',
       'secondary_cleaner.state.floatbank6_a_level']

numeric_test = ['primary_cleaner.input.sulfate',
       'primary_cleaner.input.depressant', 'primary_cleaner.input.feed_size',
       'primary_cleaner.input.xanthate',
       'primary_cleaner.state.floatbank8_a_air',
       'primary_cleaner.state.floatbank8_a_level',
       'primary_cleaner.state.floatbank8_b_air',
       'primary_cleaner.state.floatbank8_b_level',
       'primary_cleaner.state.floatbank8_c_air',
       'primary_cleaner.state.floatbank8_c_level',
       'primary_cleaner.state.floatbank8_d_air',
       'primary_cleaner.state.floatbank8_d_level', 'rougher.input.feed_ag',
       'rougher.input.feed_pb', 'rougher.input.feed_rate',
       'rougher.input.feed_size', 'rougher.input.feed_sol',
       'rougher.input.feed_au', 'rougher.input.floatbank10_sulfate',
       'rougher.input.floatbank10_xanthate',
       'rougher.input.floatbank11_sulfate',
       'rougher.input.floatbank11_xanthate', 'rougher.state.floatbank10_a_air',
       'rougher.state.floatbank10_a_level', 'rougher.state.floatbank10_b_air',
       'rougher.state.floatbank10_b_level', 'rougher.state.floatbank10_c_air',
       'rougher.state.floatbank10_c_level', 'rougher.state.floatbank10_d_air',
       'rougher.state.floatbank10_d_level', 'rougher.state.floatbank10_e_air',
       'rougher.state.floatbank10_e_level', 'rougher.state.floatbank10_f_air',
       'rougher.state.floatbank10_f_level',
       'secondary_cleaner.state.floatbank2_a_air',
       'secondary_cleaner.state.floatbank2_a_level',
       'secondary_cleaner.state.floatbank2_b_air',
       'secondary_cleaner.state.floatbank2_b_level',
       'secondary_cleaner.state.floatbank3_a_air',
       'secondary_cleaner.state.floatbank3_a_level',
       'secondary_cleaner.state.floatbank3_b_air',
       'secondary_cleaner.state.floatbank3_b_level',
       'secondary_cleaner.state.floatbank4_a_air',
       'secondary_cleaner.state.floatbank4_a_level',
       'secondary_cleaner.state.floatbank4_b_air',
       'secondary_cleaner.state.floatbank4_b_level',
       'secondary_cleaner.state.floatbank5_a_air',
       'secondary_cleaner.state.floatbank5_a_level',
       'secondary_cleaner.state.floatbank5_b_air',
       'secondary_cleaner.state.floatbank5_b_level',
       'secondary_cleaner.state.floatbank6_a_air',
       'secondary_cleaner.state.floatbank6_a_level']

# fit scaler for train and full on full data
scaler_full_train = StandardScaler()
scaler_full_train.fit(data_full.drop(['date'], axis=1)) 

# fit scaler for test on test data
scaler_test = StandardScaler()
scaler_test.fit(data_test.drop(['date'], axis=1)) 

#### hold

# scale all 3 datasets
data_full[numeric_full_train] = scaler_full_train.transform(data_full[numeric_full_train])
data_train[numeric_full_train] = scaler_full_train.transform(data_train[numeric_full_train])
data_test[numeric_test] = scaler_test.transform(data_test[numeric_test])

### Add  datetime column 

In [315]:
data_train['date_datetime'] = pd.to_datetime(data_train['date'], format='%Y-%m-%d %H:%M:%S') 
data_test['date_datetime'] = pd.to_datetime(data_test['date'], format='%Y-%m-%d %H:%M:%S') 
data_full['date_datetime'] = pd.to_datetime(data_test['date'], format='%Y-%m-%d %H:%M:%S') 

### Add target columns to the test set from the full columns base on indexes

In [316]:
# columns to add and date that is the ref column
recovery_strings = ['date', 'final.output.recovery', 'rougher.output.recovery']

In [317]:
data_test = data_test.merge(right=data_full[recovery_strings], 
                           how='left',
                           on='date')

# Analyze the data

## Take note of how the concentrations of metals (Au, Ag, Pb) change depending on the purification stage


In this section I will do the following:
- Plot histogram of the metals concentration with line for each purification stage 
- Explain the graph
- Using only training set for this task because it contains the metals concentration as oppose to the test set that it's absent there 

In [318]:
# get only column name with ag au pb and date 
matching = [
    s for s in data_train.columns.to_list() if 'au' in s or 'ag' in s or 'pb' in s or 'datetime' in s]
matching

['final.output.concentrate_ag',
 'final.output.concentrate_pb',
 'final.output.concentrate_au',
 'final.output.tail_ag',
 'final.output.tail_pb',
 'final.output.tail_au',
 'primary_cleaner.output.concentrate_ag',
 'primary_cleaner.output.concentrate_pb',
 'primary_cleaner.output.concentrate_au',
 'primary_cleaner.output.tail_ag',
 'primary_cleaner.output.tail_pb',
 'primary_cleaner.output.tail_au',
 'rougher.calculation.sulfate_to_au_concentrate',
 'rougher.calculation.floatbank10_sulfate_to_au_feed',
 'rougher.calculation.floatbank11_sulfate_to_au_feed',
 'rougher.calculation.au_pb_ratio',
 'rougher.input.feed_ag',
 'rougher.input.feed_pb',
 'rougher.input.feed_au',
 'rougher.output.concentrate_ag',
 'rougher.output.concentrate_pb',
 'rougher.output.concentrate_au',
 'rougher.output.tail_ag',
 'rougher.output.tail_pb',
 'rougher.output.tail_au',
 'secondary_cleaner.output.tail_ag',
 'secondary_cleaner.output.tail_pb',
 'secondary_cleaner.output.tail_au',
 'date_datetime']

In [319]:
# remove calculation and tail strings
matching = [s for s in matching if ('tail' not in s) and ('calculation' not in s)]
matching

['final.output.concentrate_ag',
 'final.output.concentrate_pb',
 'final.output.concentrate_au',
 'primary_cleaner.output.concentrate_ag',
 'primary_cleaner.output.concentrate_pb',
 'primary_cleaner.output.concentrate_au',
 'rougher.input.feed_ag',
 'rougher.input.feed_pb',
 'rougher.input.feed_au',
 'rougher.output.concentrate_ag',
 'rougher.output.concentrate_pb',
 'rougher.output.concentrate_au',
 'date_datetime']

In [320]:
# sort by stage order the ag
matching_ag = [
 'date_datetime',
 'rougher.input.feed_ag',
 'rougher.output.concentrate_ag',
 'primary_cleaner.output.concentrate_ag',
 'final.output.concentrate_ag'
]

# sort by stage order the pb
matching_pb = [
 'date_datetime',
 'rougher.input.feed_pb',
 'rougher.output.concentrate_pb',
 'primary_cleaner.output.concentrate_pb',
 'final.output.concentrate_pb'
]

# sort by stage order the au
matching_au = [
 'date_datetime',
 'rougher.input.feed_au',
 'rougher.output.concentrate_au',
 'primary_cleaner.output.concentrate_au',
 'final.output.concentrate_au'
]

#### hold

# produce hist for ag features for the train df
for column in matching_ag:
    px.histogram(data_train[matching_ag], x=column).show()

- The ag concentration increases in the flotation stage. 
- In 2016 the concentration increased from ~8 to ~13 in flotation stage. 
- In the two later stage the concentration decreased a little bit. 
- It is clear the process does not increase the concentration of ag in the process line as should be expected since this is not a process to extract ag.

#### hold

# produce hist for pb features for the train df
for column in matching_pb:
    px.histogram(data_train[matching_pb], x=column).show()

- pb concentration increased in the flotation from ~3 to ~8
- After that in the last two stages it increased a bit up to ~10
- This is because this process is not focus on pb

#### hold

# produce hist for au features for the train df
for column in matching_au:
    px.histogram(data_train[matching_au], x=column).show()

- The au (gold) concentration increases in **all** stages. 
- The concentration increased from ~7 to ~20 in flotation stage. 
- In the primary cleaner the concentration increased to ~34.
- In the last stage the concentration increased to ~45!

### concentrations of metals conclusion
It is clear that the process is selective to gold. The other metals concentration almost don't change and for some of the cases it decreased. This show that for purifying the gold from the ore the other metals removed.


## Compare the feed particle size distributions in the training set and in the test set. If the distributions vary significantly, the model evaluation will be incorrect.


In this section I will do the following:

- Produce histograms of the particle size distributions in the training set and in the test set.
- Run t-test 
- If distributions are similar, it is safe to use the test to evaluate the trained model


In [321]:
# Get the columns of size distribution
size_columns = [
    s for s in data_test.columns.to_list() if 'size' in s]
size_columns

['primary_cleaner.input.feed_size', 'rougher.input.feed_size']

- I will take only the rougher input because this is not affected by the process and our goal is only to determine if something out of the process happen different in times it was taken for test or for training so that there is different

In [322]:
# hisatogram of test set rougher.input.feed_size
px.histogram(data_test, x='rougher.input.feed_size').show()

In [323]:
# hisatogram of train set rougher.input.feed_size
px.histogram(data_train, x='rougher.input.feed_size').show()

Conduct t-test to compare the mean of the two groups

We will test the following hypothesises:

- The means particle size in the test and train set are equal to each other.
 - The null hypothesis H<sub>0</sub> is that the means are NOT equal to each other.
 - The alternative hypothesis H<sub>1</sub> is that the means are equal to each other.

In [324]:
alpha = 0.05  # critical statistical significance level
# if the p-value is less than alpha, we reject the hypothesis

results = stats.ttest_ind(
    data_train['rougher.input.feed_size'],
    data_test['rougher.input.feed_size'])

print('p-value: ', results.pvalue)

if results.pvalue < alpha:
    print("We reject the null hypothesis the means are equal to each other")
else:
    print("We can't reject the null hypothesis the means are NOT equal to each other") 

p-value:  1.7978747359729643e-17
We reject the null hypothesis the means are equal to each other


The particle size are similar, it is safe to use the test to evaluate the trained model

## Consider the total concentrations of all substances at different stages: raw feed, rougher concentrate, and final concentrate. Do you notice any abnormal values in the total distribution? If you do, is it worth removing such values from both samples? Describe the findings and eliminate anomalies.


To solve this one I will do the following

- All substances will be Au, Ag, Pb. 
- Combine their concentration in each stage to produce a new column called:       [stage].total_concentration
- Produce histogram
- Look for abnormalities such as:
 - In later stages the amount increases
- Do this only for the train because in the test concentrations not appear


In [325]:
# create column  for each of the process stage
rougher_input_feed = [
 'rougher.input.feed_ag',
 'rougher.input.feed_pb',
 'rougher.input.feed_au'
]

rougher_output_concentrate = [
 'final.output.concentrate_ag',
 'final.output.concentrate_pb',
 'final.output.concentrate_au'
]

primary_cleaner_output_concentrate = [
 'primary_cleaner.output.concentrate_ag',
 'primary_cleaner.output.concentrate_pb',
 'primary_cleaner.output.concentrate_au'
]

final_output_concentrate = [
 'final.output.concentrate_ag',
 'final.output.concentrate_pb',
 'final.output.concentrate_au'
]

column_list = [
    'rougher_input_feed',
    'rougher_output_concentrate',
    'primary_cleaner_output_concentrate',
    'final_output_concentrate'
]

In [326]:
# create df for only substance
df_for_ag_au_pb_total = data_train[[
 'final.output.concentrate_ag',
 'final.output.concentrate_pb',
 'final.output.concentrate_au',
 'primary_cleaner.output.concentrate_ag',
 'primary_cleaner.output.concentrate_pb',
 'primary_cleaner.output.concentrate_au',
 'rougher.input.feed_ag',
 'rougher.input.feed_pb',
 'rougher.input.feed_au',
 'rougher.output.concentrate_ag',
 'rougher.output.concentrate_pb',
 'rougher.output.concentrate_au'
]]

In [327]:
# sum concentrations by stage
df_for_ag_au_pb_total['rougher_input_feed'] = df_for_ag_au_pb_total[
    rougher_input_feed].sum(axis=1) 
df_for_ag_au_pb_total['rougher_output_concentrate']= df_for_ag_au_pb_total[
    rougher_output_concentrate].sum(axis=1)
df_for_ag_au_pb_total['primary_cleaner_output_concentrate']= df_for_ag_au_pb_total[
    primary_cleaner_output_concentrate].sum(axis=1)
df_for_ag_au_pb_total['final_output_concentrate']= df_for_ag_au_pb_total[
    final_output_concentrate].sum(axis=1)

In [328]:
# left only sum
df_for_ag_au_pb_total = df_for_ag_au_pb_total[column_list]
df_for_ag_au_pb_total

,rougher_input_feed,rougher_output_concentrate,primary_cleaner_output_concentrate,final_output_concentrate
0,14.871440,58.137072,53.111627,58.137072
1,14.905729,58.699942,53.174338,58.699942
2,14.638299,58.927421,52.927464,58.927421
3,14.199305,58.714815,43.979185,58.714815
4,13.511497,59.065251,52.121146,59.065251
...,...,...,...,...
16855,16.045275,61.295107,51.945355,61.295107
16856,15.104931,61.412113,52.161648,61.412113
16857,14.515579,61.340054,51.619196,61.340054
16858,14.313448,61.401016,50.273714,61.401016


#### hold

# plot histograms

# Group data together
hist_data = [df_for_ag_au_pb_total['rougher_input_feed'],
             df_for_ag_au_pb_total['rougher_output_concentrate'],
             df_for_ag_au_pb_total['primary_cleaner_output_concentrate'],
             df_for_ag_au_pb_total['final_output_concentrate']]
        

group_labels = ['rougher_input_feed',
                'rougher_output_concentrate',
                'primary_cleaner_output_concentrate',
                'final_output_concentrate']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)
fig.show()

the findings are:
- The total concentration increase from the feed to the output of the primary cleaner and is the highest in the final output
- The abnormality is that after the flotation (first stage) total concentration and final the total concentrations are appear to be the same. This make no sense because after the flotation comes the primary cleaner where the concentration decreases and then return back up to look exactly the same in the histogram
\
\
\
I will not take any action about it now because it does not affect the fact that the gold concentration itself behaves as expected and reduced in every step of the way

# Build the model

## Write a function to calculate the final sMAPE value.

What I will do here is:
- Write a function to calculate the sMAPE for the rougher and the final stage using the predicted recovery and target recovery
- test that the function works using the test set

In [329]:
def smape(y_true, y_pred):
    return (np.abs(y_true-y_pred)/((np.abs(y_true)+np.abs(y_pred))/2)).mean()

In [330]:
true = np.array([1,1])
pred = np.array([0,1])

In [331]:
true

array([1, 1])

In [332]:
target_test[:,0]

array([89.99342112, 88.0896569 , 88.41275562, ..., 93.01813839,
       92.59904222, 91.1776948 ])

In [333]:
smape(true,pred)

1.0

First we will create features and target sets for the training set and for the test set 

In [334]:
features_train = data_train[data_test.columns]
features_train = features_train.drop(['final.output.recovery',
                                      'rougher.output.recovery',
                                     'date',
                                     'date_datetime'], axis=1).values
target_train = data_train[['rougher.output.recovery', 'final.output.recovery']].values

In [335]:
features_train.shape

(16860, 52)

In [336]:
target_train.shape

(16860, 2)

In [337]:
features_test = data_test.drop(['final.output.recovery',
                                      'rougher.output.recovery',
                                     'date',
                                     'date_datetime'], axis=1).values
target_test = data_test[['rougher.output.recovery', 'final.output.recovery']].values

In [338]:
target_test.shape

(5856, 2)

In [339]:
target_test.shape

(5856, 2)

In [340]:
lr = LinearRegression()
lr.fit(features_train, target_train)

LinearRegression()

In [341]:
pred_lr = lr.predict(features_test)

In [342]:
smape(target_test[:,0], pred_lr[:,0])

0.11398862946060925

In [343]:
smape(target_test[:,1], pred_lr[:,1])

0.14890982583006288

In [344]:
def smape_final(y_true,y_pred):
    smape_out_rougher = smape(y_true[:,0], y_pred[:,0])
    smape_out_final = smape(y_true[:,1], y_pred[:,1])
    return 0.25*smape_out_rougher + 0.75*smape_out_final

In [345]:
smape_final(target_test, pred_lr)

0.14017952673769948

In [346]:
r

NameError: name 'r' is not defined

## Train different models. Evaluate them using cross-validation. Pick the best model and test it using the test sample. Provide findings

In [ ]:
smape_score = make_scorer(smape_final)

In [ ]:
scores = cross_val_score(estimator=lr,
                         X=features_train,
                         y=target_train,
                         cv=5,
                         scoring=smape_score)

In [ ]:
scores

In [ ]:
scores.mean()

In [ ]:
# create a function to run the evaluation automaticaly
def cv_evaluation(model):
    smape_score = make_scorer(smape_final)
    scores = cross_val_score(model, features_train, target_train, cv=5, scoring=smape_score)
    print('Mean smape: {:.2f}. Folds: {}'.format(scores.mean(), scores))

In [ ]:
# linear regression
cv_evaluation(lr)

In [ ]:
# random forest
rfr = RandomForestRegressor(n_estimators=50)
rfr.fit(features_train, target_train)

In [ ]:
cv_evaluation(rfr)

How to use GridSearchCV?